In [13]:
# import sys
import torch
import numpy as np
sys.path.append('../')
from wbident import PSO, Model
import matplotlib.pyplot as plt

In [14]:
class EqSystem(Model):
    def __init__(self, params=None):
        super(EqSystem, self).__init__(params)
        self._params = params

    def model(self, t, x):
        def delta(vel):
            if abs(vel) > 0.1:
                d = 15
            else:
                d = 1.5
            return d
        k = self.unknown_const
        ks   = k[0]
        c    = k[1]
        m    = 1
        wn   = np.sqrt(k[0]/m)
        zeta = k[1]/(2*m*wn)
        dx = torch.zeros(len(self.x0),)
        dx[0] = x[1]
        dx[1] = -2 * zeta * wn * delta(x[1])*x[1] - wn ** 2 * x[0] + np.sin(2*np.pi*0.5*t)
        return dx


In [15]:
params = {'optmizer': {'lowBound': [0.5, 0.5],
                        'upBound': [15, 10],
                        'maxVelocity': 2, 
                        'minVelocity': -2,
                        'nPop': 10,
                        'nVar': 2,
                        'social_weight': 2,
                        'cognitive_weight': 1,
                        'w': 0.9,
                        'w_damping': 0.99},
            'dyn_system': {'model_path': '',
                            'x0': [0., 0.],
                            't': [0,6,1000]
                            }
            }

In [17]:
f_fit = EqSystem(params)
k = torch.tensor([120,5])
f_fit.y = f_fit.simulation(k)
pso = PSO(f_fit, params)
cost = []


for i in range(100):
    print(f'interation {i}, error {pso.pbg_cost},{pso.pbg_position}')
    pso.run()
    print(f'interation {i} {pso.pbg_cost},{pso.pbg_position}')


interation 0, error inf,tensor([8.9456, 9.0311])


KeyboardInterrupt: 